In [2]:
import datetime as dt
from datetime import date
import os
import sys
import numpy as np
import pandas as pd
from glob import glob
import sqlalchemy
import pymysql
import time, datetime
from datetime import datetime, timedelta
from datetime import timedelta, date
import time
from numpy import sort

In [27]:
base = pd.read_csv(r"D:\analise_corr()\analise_dummy_ou_outros (version 1).csv", sep=';', low_memory=False)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe7 in position 240: invalid continuation byte

In [17]:
import re


def somase(data_frame, pv_pai, flag_cont):
        nova_coluna = data_frame.groupby(pv_pai)[flag_cont].sum()
        data_frame = data_frame.merge(nova_coluna, left_on=pv_pai, right_on=pv_pai,how='left')

        return data_frame




base['flag_teste'] = base.apply(
    lambda row: somase(row['pv_pai'], row['Flag Atraso Transporte']), axis=1)

base.renme[] 




TypeError: 'int' object is not iterable